In [1]:
# numerical computation
import numpy as np
import itertools
from random import randint

# import sklearn

from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
from sklearn.model_selection import GridSearchCV

# dataframe management
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

from IPython.display import clear_output

%autosave 60


Autosaving every 60 seconds


In [2]:
def opened (path=''):
    
    X_training=[]
    X_testing=[]
    y_training=[]
    y_testing=[]
           
    for j in range(0, 50):
        X_training.append(pd.read_csv('test_train_dataset{}{}_X_train.csv'.format(path,j)))
        X_testing.append(pd.read_csv('test_train_dataset{}{}_X_test.csv'.format(path, j)))
        y_training.append(pd.read_csv('test_train_dataset{}{}_y_train.csv'.format(path, j)))
        y_testing.append(pd.read_csv('test_train_dataset{}{}_y_test.csv'.format(path, j)))
        
    return X_training, X_testing, y_training, y_testing

In [3]:
def frequency (valor):
    max = 0
    res = list(valor)[0] 
    for i in list(valor): 
        freq = list(valor).count(i) 
        if freq > max: 
            max = freq 
            res = i 
    valor = res
    return valor

In [4]:
def maximun (df, name):
    maximun = df.sort_values(by='accuracy_model',ascending=False).head(n=1)
    best = list(maximun[name])[0]
    return best

#  Aplicación del algoritmo RandomForest

In [5]:
def hyper_RF(path, features, name, multiclass=False):
    
    x_train, x_test, y_train, y_test = opened(path=path)
    print('Terminada la apertura de BBDD')
    
    hyper= pd.read_csv('results/DT/DT_hyper_{}.csv'.format(name))
    Depth = (maximun(hyper, 'max_depth'))
    Split = (maximun(hyper, 'min_samples_split'))
    
    turn_n_estimators = turn_n_estimators = [i for i in range(2,204)]
            
    param_grid = [
            {
                'n_estimators': turn_n_estimators
            }
           ]

    RF_evaluate=[]
    RF_acc_model=[]
    RF_error=[]
    RF_std=[]


    mean=[]
    std=[]
    best_estimator=[]
    
    
    
    for j in range(0, 50):
        droping=pd.concat([x_train[j][features], y_train[j]], axis=1,sort=False)
        droping=droping.drop_duplicates(subset=features, keep=False)
        xtrain= droping[features]
        if multiclass==True:
            ytrain=droping['CRG']
        else:
            ytrain=droping[['HP', 'Diabetes', 'Otros']]
                
        print('Particion: ', j)

    #Normalizamos x_test y x_train con la misma media y variancia que x_train
        ss=StandardScaler()
        ss.fit(xtrain)
        ss_train=ss.transform(xtrain)

    #Buscamos los mejores parametros para esa división normalizada
        clf = GridSearchCV(RandomForestClassifier(criterion='entropy',max_depth=Depth,
                                                  min_samples_split= Split), param_grid, 
                           scoring='accuracy',cv=KFold(n_splits=5), n_jobs=-1)
        
        if multiclass==True:
            y_training = ytrain.values.ravel()
        else:
            y_training = ytrain
        
        clf.fit(ss_train,y_training)

    #Evaluamos el algortimo teniendo en cuenta que para la función GridSearchCV test es nuestro train
        best_index_Acc = np.nonzero(clf.cv_results_['rank_test_score'] == 1)[0][0]
        
        best_estimator.append(clf.best_params_['n_estimators'])

        RF_acc_model.append(clf.cv_results_['mean_test_score'][best_index_Acc])
        RF_std.append(clf.cv_results_['std_test_score'][best_index_Acc])

        RF_evaluate.append([best_estimator[j], round(RF_acc_model[j],3), round(RF_std[j],3)])
    
             
    labels_comp = ['n_estimators', 'accuracy_model', 'std']

    comparacion=pd.DataFrame(data=RF_evaluate, columns = labels_comp)

    comparacion.to_csv('results/RF/RF_hyper_{}.csv'.format(name), index=False)

    Estimators = (maximun(comparacion, 'n_estimators'))
    print('Estimators: ', Estimators)

In [6]:
def predict_RF(path, features, name, multiclass=False):

    
    x_train, x_test, y_train, y_test = opened(path=path)
    print('Terminada la apertura de BBDD')
    
    hyper= pd.read_csv('results/DT/DT_hyper_{}.csv'.format(name))
    Depth = (maximun(hyper, 'max_depth'))
    Split = (maximun(hyper, 'min_samples_split'))
    
    hyper= pd.read_csv('results/RF/RF_hyper_{}.csv'.format(name))
    Estimators = (maximun(hyper, 'n_estimators'))
    print('Estimators: ', Estimators)
    
    accuracy=[]
    hamming_losse=[]
    precision_macro=[]
    precision_micro=[]
    recall_macro=[]
    recall_micro=[]
    f1_scores_macro=[]
    f1_scores_micro=[]
    
    average_accuracy=[]
    average_precision=[]
    average_recall=[]
    f1_scores=[]
    
    for i in range(0,50):
        droping_train=pd.concat([x_train[i][features], y_train[i]], axis=1,sort=False)
        droping_train=droping_train.drop_duplicates(subset=features, keep=False)
        xtrain= droping_train[features]
        if multiclass==True:
            ytrain=droping_train['CRG']
        else:
            ytrain=droping_train[['HP', 'Diabetes', 'Otros']]

        droping_test=pd.concat([x_test[i][features], y_test[i]], axis=1,sort=False)
        droping=droping_test.drop_duplicates(subset=features, keep=False)
        xtest= droping_test[features]
        if multiclass==True:
            ytest=droping_test['CRG']
        else:
            ytest=droping_test[['HP', 'Diabetes', 'Otros']]
                
        ss=StandardScaler()
        ss.fit(xtrain)
        ss_train=ss.transform(xtrain)
        ss_test=ss.transform(xtest)

        clf= RandomForestClassifier(criterion='entropy',max_depth=Depth, 
                                    min_samples_split= Split, 
                                    n_estimators=Estimators)
        
        if multiclass==True:
            y_training = ytrain.values.ravel()
        else:
            y_training = ytrain
               
        clf.fit(ss_train,y_training)
        
        y_true, y_pred = ytest, clf.predict(ss_test)
        
        if multiclass==False:
            accuracy.append(accuracy_score(y_true, y_pred))
            hamming_losse.append(hamming_loss(y_true, y_pred))
            precision_macro.append(precision_score(y_true,y_pred, average='macro'))
            precision_micro.append(precision_score(y_true,y_pred, average='micro'))
            recall_macro.append(recall_score(y_true, y_pred, average='macro'))
            recall_micro.append(recall_score(y_true, y_pred, average='micro'))
            f1_scores_macro.append(f1_score(y_true, y_pred, average='macro'))
            f1_scores_micro.append(f1_score(y_true, y_pred, average='micro'))
        else:
            cm = confusion_matrix(y_true,y_pred)
            TP = np.diag(cm)
            FP = np.sum(cm, axis=0) - TP
            FN = np.sum(cm, axis=1) - TP
            #num_classes = len(TP)
            #TN = []
            #for i in range(num_classes):
            #    temp = np.delete(cm, i, 0)    # delete ith row
            #    temp = np.delete(temp, i, 1)  # delete ith column
            #    TN.append(sum(sum(temp)))

            precision=TP / (TP + FP)
            recall=TP / (TP + FN)

            average_precision.append(np.mean(TP / (TP + FP)))
            average_recall.append(np.mean(TP / (TP + FN)))
            f1_scores.append(np.mean(2*(precision*recall)/(precision+recall)))
            average_accuracy.append(accuracy_score(y_true, y_pred))
        
    predict=pd.DataFrame()
    if multiclass==False:
        predict['accuracy']=accuracy
        predict['hamming_loss'] = hamming_losse
        predict['precision_macro']=precision_macro
        predict['precision_micro']=precision_micro
        predict['recall_macro']=recall_macro
        predict['recall_micro']=recall_micro
        predict['f1_macro']=f1_scores_macro
        predict['f1_micro']=f1_scores_micro
    else:
        predict['accuracy']=average_accuracy
        predict['precision']=average_precision
        predict['recall']=average_recall
        predict['f1']=f1_scores
        
    predict.to_csv('results/RF/RF_predict_{}.csv'.format(name), index=False)


## Bucles para las diferentes ejecuciones

In [7]:
import os.path as path

### Selección de Caracteriticas: Frecuencia

In [8]:
names = ['ocurrencia_all', 'ocurrencia_ill', 'presencia_all', 'presencia_ill'] 
features_freq = []
for n in names:
    with open("feature_selection/freq_{}.txt".format(n), "r") as file:
        features_freq.append(eval(file.readline()))

In [9]:
paths_CLASS = ['/class/O_WC_A_','/class/O_WC_WO_' , '/class/P_WC_A_', '/class/P_WC_WO_']
names_CLASS_fr=['freq_all_class_O', 'freq_ill_class_O', 'freq_all_class_P', 'freq_ill_class_P']

In [10]:
for p, n, f in zip(paths_CLASS, names_CLASS_fr, features_freq):
    if path.exists('results/RF/RF_hyper_{}.csv'.format(n)): 
        print('Ya existe el hyperparametro:', n)
    else:
        hyper_RF(p, f, n, True)
        print()
        print('--------------------------------------------------------')
        print()
    
    if path.exists('results/RF/RF_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_RF(p, f, n, True)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: freq_all_class_O
Ya existe los resultados: freq_all_class_O
Ya existe el hyperparametro: freq_ill_class_O
Ya existe los resultados: freq_ill_class_O
Ya existe el hyperparametro: freq_all_class_P
Ya existe los resultados: freq_all_class_P
Ya existe el hyperparametro: freq_ill_class_P
Ya existe los resultados: freq_ill_class_P


In [11]:
paths_LABEL = ['/label/O_WL_A_','/label/O_WL_WO_' , '/label/P_WL_A_', '/label/P_WL_WO_']
names_LABEL_fr=['freq_all_label_O', 'freq_ill_label_O', 'freq_all_label_P', 'freq_ill_label_P']

In [12]:
for p, n, f in zip(paths_LABEL, names_LABEL_fr, features_freq):
    if path.exists('results/RF/RF_hyper_{}.csv'.format(n)): 
        print('Ya existe el hyperparametro:', n)
    else:
        hyper_RF(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()
    
    if path.exists('results/RF/RF_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_RF(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: freq_all_label_O
Ya existe los resultados: freq_all_label_O
Ya existe el hyperparametro: freq_ill_label_O
Ya existe los resultados: freq_ill_label_O
Ya existe el hyperparametro: freq_all_label_P
Ya existe los resultados: freq_all_label_P
Ya existe el hyperparametro: freq_ill_label_P
Ya existe los resultados: freq_ill_label_P


### Selección de Caracteriticas: Random Forest

In [13]:
names=['label_o_all','label_o_ill', 'label_p_all', 'label_p_ill']
features_rf_label = []
for n in names:
    with open("feature_selection/rf_{}.txt".format(n), "r") as file:
        features_rf_label.append(eval(file.readline()))

In [14]:
path_label= ['/label/O_WL_A_', '/label/O_WL_WO_', '/label/P_WL_A_', '/label/P_WL_WO_']
names_label_rf=['rf_all_label_O','rf_ill_label_O', 'rf_all_label_P', 'rf_ill_label_P']

In [15]:
for p, n, f in zip(path_label, names_label_rf, features_rf_label):
    if path.exists('results/RF/RF_hyper_{}.csv'.format(n)): 
        print('Ya existe el hyperparametro:', n)
    else:
        hyper_RF(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()
    
    if path.exists('results/RF/RF_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_RF(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: rf_all_label_O
Ya existe los resultados: rf_all_label_O
Ya existe el hyperparametro: rf_ill_label_O
Ya existe los resultados: rf_ill_label_O
Ya existe el hyperparametro: rf_all_label_P
Ya existe los resultados: rf_all_label_P
Ya existe el hyperparametro: rf_ill_label_P
Ya existe los resultados: rf_ill_label_P


In [16]:
names=['class_o_all','class_o_ill', 'class_p_all', 'class_p_ill']
features_rf_class = []
for n in names:
    with open("feature_selection/rf_{}.txt".format(n), "r") as file:
        features_rf_class.append(eval(file.readline()))

In [17]:
path_class= ['/class/O_WC_A_', '/class/O_WC_WO_', '/class/P_WC_A_', '/class/P_WC_WO_']
name_class_rf=['rf_all_class_O','rf_ill_class_O', 'rf_all_class_P', 'rf_ill_class_P']

In [18]:
for p, n, f in zip(path_class, name_class_rf, features_rf_class):  
    if path.exists('results/RF/RF_hyper_{}.csv'.format(n)): 
        print('Ya existe el hyperparametro:', n)
    else:
        hyper_RF(p, f, n, multiclass=True)
        print()
        print('--------------------------------------------------------')
        print()
    
    if path.exists('results/RF/RF_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_RF(p, f, n, multiclass=True)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: rf_all_class_O
Ya existe los resultados: rf_all_class_O
Ya existe el hyperparametro: rf_ill_class_O
Ya existe los resultados: rf_ill_class_O
Ya existe el hyperparametro: rf_all_class_P
Ya existe los resultados: rf_all_class_P
Ya existe el hyperparametro: rf_ill_class_P
Ya existe los resultados: rf_ill_class_P


### Seleccion de frecuencia: F Fisher

In [19]:
names=['label_o_all','label_o_ill', 'label_p_all', 'label_p_ill']
features_fc_label = []
for n in names:
    with open("feature_selection/fc_{}.txt".format(n), "r") as file:
        features_fc_label.append(eval(file.readline()))

In [20]:
path_label= ['/label/O_WL_A_', '/label/O_WL_WO_', '/label/P_WL_A_', '/label/P_WL_WO_']
names_label_fc=['fc_all_label_O','fc_ill_label_O', 'fc_all_label_P', 'fc_ill_label_P']

In [21]:
for p, n, f in zip(path_label, names_label_fc, features_fc_label):
    if path.exists('results/RF/RF_hyper_{}.csv'.format(n)): 
        print('Ya existe el hyperparametro:', n)
    else:
        hyper_RF(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()
    
    if path.exists('results/RF/RF_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_RF(p, f, n)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: fc_all_label_O
Ya existe los resultados: fc_all_label_O
Ya existe el hyperparametro: fc_ill_label_O
Ya existe los resultados: fc_ill_label_O
Ya existe el hyperparametro: fc_all_label_P
Ya existe los resultados: fc_all_label_P
Ya existe el hyperparametro: fc_ill_label_P
Ya existe los resultados: fc_ill_label_P


In [22]:
names=['class_o_all','class_o_ill', 'class_p_all', 'class_p_ill']
features_rf_class = []
for n in names:
    with open("feature_selection/rf_{}.txt".format(n), "r") as file:
        features_rf_class.append(eval(file.readline()))

In [23]:
path_class= ['/class/O_WC_A_', '/class/O_WC_WO_', '/class/P_WC_A_', '/class/P_WC_WO_']
name_class_fc=['fc_all_class_O','fc_ill_class_O', 'fc_all_class_P', 'fc_ill_class_P']

In [24]:
for p, n, f in zip(path_class, name_class_fc, features_rf_class):
    if path.exists('results/RF/RF_hyper_{}.csv'.format(n)): 
        print('Ya existe el hyperparametro:', n)
    else:
        hyper_RF(p, f, n, multiclass=True)
        print()
        print('--------------------------------------------------------')
        print()
    
    if path.exists('results/RF/RF_predict_{}.csv'.format(n)): 
        print('Ya existe los resultados:', n)
    else:
        predict_RF(p, f, n, multiclass=True)
        print()
        print('--------------------------------------------------------')
        print()

Ya existe el hyperparametro: fc_all_class_O
Ya existe los resultados: fc_all_class_O
Ya existe el hyperparametro: fc_ill_class_O
Ya existe los resultados: fc_ill_class_O
Ya existe el hyperparametro: fc_all_class_P
Ya existe los resultados: fc_all_class_P
Ya existe el hyperparametro: fc_ill_class_P
Ya existe los resultados: fc_ill_class_P


## Resultados

In [25]:
def resultados_etiqueta(names):
    hyper_label=[]
    predict_label=[]
    for name in names:
        hyper_label.append(pd.read_csv('results/RF/RF_hyper_{}.csv'.format(name)))
        predict_label.append(pd.read_csv('results/RF/RF_predict_{}.csv'.format(name)))
    for i, n in zip(range(0, len(names)), names):
        print(n)
        print()
        Estimators = (maximun(hyper_label[i], 'n_estimators'))
        print('Estimators: ', Estimators)

        print('Tasa de acierto:', round(np.mean(predict_label[i]['accuracy']), 3), '+/-', round(np.std(predict_label[i]['accuracy']), 3))
        print('Tasa de Hamming Loss:', round(np.mean(predict_label[i]['hamming_loss']), 3), '+/-', round(np.std(predict_label[i]['hamming_loss']), 3))
        print('Tasa de precision(macro)', round(np.mean(predict_label[i]['precision_macro']), 3), '+/-', round(np.std(predict_label[i]['precision_macro']), 3))
        print('Tasa de precision(micro)', round(np.mean(predict_label[i]['precision_micro']), 3), '+/-', round(np.std(predict_label[i]['precision_micro']), 3))
        print('Tasa de exactitud(macro):', round(np.mean(predict_label[i]['recall_macro']), 3),  '+/-', round(np.std(predict_label[i]['recall_macro']), 3))
        print('Tasa de exactitud(micro):', round(np.mean(predict_label[i]['recall_micro']), 3),  '+/-', round(np.std(predict_label[i]['recall_micro']), 3))
        print('Tasa F1-Score(macro)', round(np.mean(predict_label[i]['f1_macro']), 3) , '+/-', round(np.std(predict_label[i]['f1_macro']),3))
        print('Tasa F1-Score(micro)', round(np.mean(predict_label[i]['f1_micro']), 3) , '+/-', round(np.std(predict_label[i]['f1_micro']),3))
        print('---------------------------------------------------------------')

In [26]:
resultados_etiqueta(names_LABEL_fr)

freq_all_label_O

Estimators:  50
Tasa de acierto: 0.847 +/- 0.014
Tasa de Hamming Loss: 0.054 +/- 0.005
Tasa de precision(macro) 0.945 +/- 0.013
Tasa de precision(micro) 0.96 +/- 0.007
Tasa de exactitud(macro): 0.852 +/- 0.016
Tasa de exactitud(micro): 0.924 +/- 0.008
Tasa F1-Score(macro) 0.888 +/- 0.012
Tasa F1-Score(micro) 0.942 +/- 0.005
---------------------------------------------------------------
freq_ill_label_O

Estimators:  80
Tasa de acierto: 0.825 +/- 0.018
Tasa de Hamming Loss: 0.062 +/- 0.007
Tasa de precision(macro) 0.948 +/- 0.013
Tasa de precision(micro) 0.96 +/- 0.008
Tasa de exactitud(macro): 0.862 +/- 0.017
Tasa de exactitud(micro): 0.933 +/- 0.008
Tasa F1-Score(macro) 0.895 +/- 0.013
Tasa F1-Score(micro) 0.946 +/- 0.006
---------------------------------------------------------------
freq_all_label_P

Estimators:  40
Tasa de acierto: 0.833 +/- 0.013
Tasa de Hamming Loss: 0.059 +/- 0.005
Tasa de precision(macro) 0.939 +/- 0.013
Tasa de precision(micro) 0.955 +/- 0.0

In [27]:
resultados_etiqueta(names_label_fc)

fc_all_label_O

Estimators:  100
Tasa de acierto: 0.857 +/- 0.012
Tasa de Hamming Loss: 0.05 +/- 0.004
Tasa de precision(macro) 0.955 +/- 0.012
Tasa de precision(micro) 0.965 +/- 0.006
Tasa de exactitud(macro): 0.86 +/- 0.014
Tasa de exactitud(micro): 0.928 +/- 0.007
Tasa F1-Score(macro) 0.897 +/- 0.01
Tasa F1-Score(micro) 0.946 +/- 0.005
---------------------------------------------------------------
fc_ill_label_O

Estimators:  95
Tasa de acierto: 0.842 +/- 0.017
Tasa de Hamming Loss: 0.056 +/- 0.006
Tasa de precision(macro) 0.958 +/- 0.013
Tasa de precision(micro) 0.968 +/- 0.008
Tasa de exactitud(macro): 0.869 +/- 0.014
Tasa de exactitud(micro): 0.936 +/- 0.007
Tasa F1-Score(macro) 0.904 +/- 0.011
Tasa F1-Score(micro) 0.951 +/- 0.005
---------------------------------------------------------------
fc_all_label_P

Estimators:  100
Tasa de acierto: 0.855 +/- 0.015
Tasa de Hamming Loss: 0.051 +/- 0.005
Tasa de precision(macro) 0.953 +/- 0.011
Tasa de precision(micro) 0.963 +/- 0.006
Ta

In [28]:
resultados_etiqueta(names_label_rf)

rf_all_label_O

Estimators:  37
Tasa de acierto: 0.855 +/- 0.013
Tasa de Hamming Loss: 0.051 +/- 0.005
Tasa de precision(macro) 0.948 +/- 0.012
Tasa de precision(micro) 0.963 +/- 0.006
Tasa de exactitud(macro): 0.86 +/- 0.015
Tasa de exactitud(micro): 0.925 +/- 0.008
Tasa F1-Score(macro) 0.895 +/- 0.011
Tasa F1-Score(micro) 0.944 +/- 0.005
---------------------------------------------------------------
rf_ill_label_O

Estimators:  66
Tasa de acierto: 0.827 +/- 0.017
Tasa de Hamming Loss: 0.061 +/- 0.006
Tasa de precision(macro) 0.955 +/- 0.012
Tasa de precision(micro) 0.961 +/- 0.007
Tasa de exactitud(macro): 0.863 +/- 0.015
Tasa de exactitud(micro): 0.934 +/- 0.007
Tasa F1-Score(macro) 0.897 +/- 0.012
Tasa F1-Score(micro) 0.947 +/- 0.005
---------------------------------------------------------------
rf_all_label_P

Estimators:  72
Tasa de acierto: 0.833 +/- 0.013
Tasa de Hamming Loss: 0.058 +/- 0.005
Tasa de precision(macro) 0.953 +/- 0.01
Tasa de precision(micro) 0.959 +/- 0.006
Tas

In [29]:
def resultados_clase(names):
    hyper_class=[]
    predict_class=[]
    for name in names:
        hyper_class.append(pd.read_csv('results/RF/RF_hyper_{}.csv'.format(name)))
        predict_class.append(pd.read_csv('results/RF/RF_predict_{}.csv'.format(name)))

    for i, n in zip(range(0, len(names)), names):
        print(n)
        print()
        Estimators = (maximun(hyper_class[i], 'n_estimators'))
        print('Estimators: ', Estimators)

        print('Tasa de acierto:', round(np.mean(predict_class[i]['accuracy']), 3), '+/-', round(np.std(predict_class[i]['accuracy']), 3))
        print('Tasa de precision', round(np.mean(predict_class[i]['precision']), 3), '+/-', round(np.std(predict_class[i]['precision']), 3))
        print('Tasa de exactitud:', round(np.mean(predict_class[i]['recall']), 3),  '+/-', round(np.std(predict_class[i]['recall']), 3))
        print('Tasa F1-Score', round(np.mean(predict_class[i]['f1']), 3) , '+/-', round(np.std(predict_class[i]['f1']),3))
        print('---------------------------------------------------------------')

In [30]:
resultados_clase(names_CLASS_fr)

freq_all_class_O

Estimators:  85
Tasa de acierto: 0.853 +/- 0.014
Tasa de precision 0.854 +/- 0.014
Tasa de exactitud: 0.853 +/- 0.014
Tasa F1-Score 0.852 +/- 0.014
---------------------------------------------------------------
freq_ill_class_O

Estimators:  70
Tasa de acierto: 0.832 +/- 0.016
Tasa de precision 0.833 +/- 0.016
Tasa de exactitud: 0.832 +/- 0.016
Tasa F1-Score 0.831 +/- 0.016
---------------------------------------------------------------
freq_all_class_P

Estimators:  105
Tasa de acierto: 0.849 +/- 0.017
Tasa de precision 0.85 +/- 0.016
Tasa de exactitud: 0.849 +/- 0.017
Tasa F1-Score 0.847 +/- 0.017
---------------------------------------------------------------
freq_ill_class_P

Estimators:  95
Tasa de acierto: 0.828 +/- 0.016
Tasa de precision 0.831 +/- 0.016
Tasa de exactitud: 0.828 +/- 0.016
Tasa F1-Score 0.827 +/- 0.016
---------------------------------------------------------------


In [31]:
resultados_clase(name_class_fc)

fc_all_class_O

Estimators:  80
Tasa de acierto: 0.865 +/- 0.014
Tasa de precision 0.868 +/- 0.014
Tasa de exactitud: 0.865 +/- 0.014
Tasa F1-Score 0.864 +/- 0.014
---------------------------------------------------------------
fc_ill_class_O

Estimators:  75
Tasa de acierto: 0.848 +/- 0.017
Tasa de precision 0.851 +/- 0.018
Tasa de exactitud: 0.848 +/- 0.017
Tasa F1-Score 0.847 +/- 0.017
---------------------------------------------------------------
fc_all_class_P

Estimators:  30
Tasa de acierto: 0.853 +/- 0.017
Tasa de precision 0.855 +/- 0.016
Tasa de exactitud: 0.853 +/- 0.016
Tasa F1-Score 0.852 +/- 0.017
---------------------------------------------------------------
fc_ill_class_P

Estimators:  100
Tasa de acierto: 0.852 +/- 0.016
Tasa de precision 0.858 +/- 0.016
Tasa de exactitud: 0.852 +/- 0.016
Tasa F1-Score 0.852 +/- 0.016
---------------------------------------------------------------


In [32]:
resultados_clase(name_class_rf)

rf_all_class_O

Estimators:  81
Tasa de acierto: 0.86 +/- 0.014
Tasa de precision 0.861 +/- 0.015
Tasa de exactitud: 0.86 +/- 0.014
Tasa F1-Score 0.859 +/- 0.014
---------------------------------------------------------------
rf_ill_class_O

Estimators:  61
Tasa de acierto: 0.838 +/- 0.014
Tasa de precision 0.838 +/- 0.015
Tasa de exactitud: 0.838 +/- 0.014
Tasa F1-Score 0.836 +/- 0.015
---------------------------------------------------------------
rf_all_class_P

Estimators:  92
Tasa de acierto: 0.856 +/- 0.017
Tasa de precision 0.858 +/- 0.016
Tasa de exactitud: 0.856 +/- 0.017
Tasa F1-Score 0.855 +/- 0.017
---------------------------------------------------------------
rf_ill_class_P

Estimators:  71
Tasa de acierto: 0.842 +/- 0.016
Tasa de precision 0.845 +/- 0.016
Tasa de exactitud: 0.842 +/- 0.016
Tasa F1-Score 0.841 +/- 0.016
---------------------------------------------------------------


# Mejor resultados

## Multi-clase

In [35]:
names = ['fc_class_p_all', 'fc_class_p_ill']
names_features=['atc', 'cie', 'cie_atc']
features = []
for n in names:
    for f in names_features:
        with open("feature_selection/best/{}_{}.txt".format(n, f), "r") as file:
            features.append(eval(file.readline()))
    features += [['Edad', 'Sexo']]
paths_CLASS = ['/class/P_WC_A_', '/class/P_WC_WO_']
names_CLASS=['fc_all_class_P_atc', 'fc_all_class_P_cie', 'fc_all_class_P_cie_atc', 'fc_all_class_P_E_S', 
             'fc_ill_class_P_atc', 'fc_ill_class_P_cie', 'fc_ill_class_P_cie_atc', 'fc_ill_class_P_E_S']

In [36]:
k=['all','ill']
for p, i in zip(paths_CLASS, k):
    if i=='all':
        names = names_CLASS[0:4]
        feat = features[0:4]
    else:
        names = names_CLASS[4:8]
        feat = features[4:8]
    for n, f in zip( names, feat):
        if path.exists('results/RF/RF_hyper_{}.csv'.format(n)): 
            print('Ya existe el hyperparametro:', n)
        else:
            hyper_RF(p, f, n, True)
            print()
            print('--------------------------------------------------------')
            print()

        if path.exists('results/RF/RF_predict_{}.csv'.format(n)): 
            print('Ya existe los resultados:', n)
        else:
            predict_RF(p, f, n, True)
            print()
            print('--------------------------------------------------------')
            print()

Ya existe el hyperparametro: fc_all_class_P_atc
Ya existe los resultados: fc_all_class_P_atc
Ya existe el hyperparametro: fc_all_class_P_cie
Ya existe los resultados: fc_all_class_P_cie
Ya existe el hyperparametro: fc_all_class_P_cie_atc
Ya existe los resultados: fc_all_class_P_cie_atc
Ya existe el hyperparametro: fc_ill_class_P_atc
Ya existe los resultados: fc_ill_class_P_atc
Ya existe el hyperparametro: fc_ill_class_P_cie
Ya existe los resultados: fc_ill_class_P_cie
Ya existe el hyperparametro: fc_ill_class_P_cie_atc
Ya existe los resultados: fc_ill_class_P_cie_atc


In [37]:
resultados_clase(names_CLASS)

fc_all_class_P_atc

Estimators:  164
Tasa de acierto: 0.73 +/- 0.068
Tasa de precision 0.777 +/- 0.037
Tasa de exactitud: 0.73 +/- 0.068
Tasa F1-Score 0.716 +/- 0.081
---------------------------------------------------------------
fc_all_class_P_cie

Estimators:  44
Tasa de acierto: 0.462 +/- 0.077
Tasa de precision 0.584 +/- 0.124
Tasa de exactitud: 0.462 +/- 0.077
Tasa F1-Score 0.418 +/- 0.094
---------------------------------------------------------------
fc_all_class_P_cie_atc

Estimators:  66
Tasa de acierto: 0.876 +/- 0.014
Tasa de precision 0.88 +/- 0.013
Tasa de exactitud: 0.876 +/- 0.014
Tasa F1-Score 0.876 +/- 0.014
---------------------------------------------------------------
fc_ill_class_P_atc

Estimators:  168
Tasa de acierto: 0.771 +/- 0.06
Tasa de precision 0.798 +/- 0.03
Tasa de exactitud: 0.771 +/- 0.06
Tasa F1-Score 0.764 +/- 0.07
---------------------------------------------------------------
fc_ill_class_P_cie

Estimators:  8
Tasa de acierto: 0.349 +/- 0.088
Tasa 

## multi-label

In [38]:
names = ['fc_label_p_all', 'fc_label_p_ill']
names_features=['atc', 'cie', 'cie_atc']
features = []
for n in names:
    for f in names_features:
        with open("feature_selection/best/{}_{}.txt".format(n, f), "r") as file:
            features.append(eval(file.readline()))
    features += [['Edad', 'Sexo']]
paths_label = ['/label/P_WL_A_', '/label/P_WL_WO_']
names_label=['fc_all_label_P_atc', 'fc_all_label_P_cie', 'fc_all_label_P_cie_atc', 'fc_all_label_P_E_S', 
             'fc_ill_label_P_atc', 'fc_ill_label_P_cie', 'fc_ill_label_P_cie_atc', 'fc_ill_label_P_E_S']

In [39]:
k=['all','ill']
for p, i in zip(paths_label, k):
    if i=='all':
        names = names_label[0:4]
        feat = features[0:4]
    else:
        names = names_label[4:8]
        feat = features[4:8]
    for n, f in zip( names, feat):
        if path.exists('results/RF/RF_hyper_{}.csv'.format(n)): 
            print('Ya existe el hyperparametro:', n)
        else:
            hyper_RF(p, f, n)
            print()
            print('--------------------------------------------------------')
            print()

        if path.exists('results/RF/RF_predict_{}.csv'.format(n)): 
            print('Ya existe los resultados:', n)
        else:
            predict_RF(p, f, n)
            print()
            print('--------------------------------------------------------')
            print()

Ya existe el hyperparametro: fc_all_label_P_atc
Ya existe los resultados: fc_all_label_P_atc
Ya existe el hyperparametro: fc_all_label_P_cie
Ya existe los resultados: fc_all_label_P_cie
Ya existe el hyperparametro: fc_all_label_P_cie_atc
Ya existe los resultados: fc_all_label_P_cie_atc
Ya existe el hyperparametro: fc_all_label_P_E_S
Ya existe los resultados: fc_all_label_P_E_S
Ya existe el hyperparametro: fc_ill_label_P_atc
Ya existe los resultados: fc_ill_label_P_atc
Ya existe el hyperparametro: fc_ill_label_P_cie
Ya existe los resultados: fc_ill_label_P_cie
Ya existe el hyperparametro: fc_ill_label_P_cie_atc
Ya existe los resultados: fc_ill_label_P_cie_atc
Ya existe el hyperparametro: fc_ill_label_P_E_S
Ya existe los resultados: fc_ill_label_P_E_S


In [40]:
resultados_etiqueta(names_label)

fc_all_label_P_atc

Estimators:  176
Tasa de acierto: 0.714 +/- 0.065
Tasa de Hamming Loss: 0.104 +/- 0.023
Tasa de precision(macro) 0.908 +/- 0.025
Tasa de precision(micro) 0.885 +/- 0.045
Tasa de exactitud(macro): 0.83 +/- 0.02
Tasa de exactitud(micro): 0.897 +/- 0.012
Tasa F1-Score(macro) 0.853 +/- 0.02
Tasa F1-Score(micro) 0.89 +/- 0.021
---------------------------------------------------------------
fc_all_label_P_cie

Estimators:  160
Tasa de acierto: 0.473 +/- 0.056
Tasa de Hamming Loss: 0.198 +/- 0.025
Tasa de precision(macro) 0.73 +/- 0.042
Tasa de precision(micro) 0.734 +/- 0.037
Tasa de exactitud(macro): 0.868 +/- 0.025
Tasa de exactitud(micro): 0.909 +/- 0.02
Tasa F1-Score(macro) 0.781 +/- 0.019
Tasa F1-Score(micro) 0.812 +/- 0.017
---------------------------------------------------------------
fc_all_label_P_cie_atc

Estimators:  60
Tasa de acierto: 0.866 +/- 0.013
Tasa de Hamming Loss: 0.047 +/- 0.005
Tasa de precision(macro) 0.956 +/- 0.011
Tasa de precision(micro) 0.964